In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
today = datetime.date.today()
#yesterday = today - datetime.timedelta(days=1)
dow=today.weekday()

if dow < 5: 
    dte=5-dow
else: 
    dte=12-dow
#print(dte)
friday = str(today + datetime.timedelta( (4-today.weekday()) % 7 ))
start_date="2010-01-01"
end_date=today

#for i in range(0,len(sip)):
def st_rsi():
    df_st_rsi=pd.DataFrame()
    for i in range(0,len(sip)):
    #for i in range(0,5):
        stk=sip[i]
        dfw = yf.download(stk, start_date, end_date,interval='1wk').dropna()
        dfm = yf.download(stk, start_date, end_date,interval='1mo').dropna()
        #dfd = yf.download(stk, yesterday, today,interval='1h').dropna()
        
        stoch_rsi_w=StochRSIIndicator(close=dfw.Close,window=14,smooth1=3, smooth2=3)
        stoch_rsi_m=StochRSIIndicator(close=dfm.Close,window=14,smooth1=3, smooth2=3)
        
        efiw=ForceIndexIndicator(close=dfw.Close,volume=dfw.Volume,window=14,fillna=True)
        dfw['efi_w']=efiw.force_index()
    
        rsiw=RSIIndicator(close=dfw.efi_w,window=14,fillna=True)
        dfw['efi_rsi_w']=rsiw.rsi()
        
        efim=ForceIndexIndicator(close=dfm.Close,volume=dfm.Volume,window=14,fillna=True)
        dfm['efi_m']=efim.force_index()
    
        rsim=RSIIndicator(close=dfm.efi_m,window=14,fillna=True)
        dfm['efi_rsi_m']=rsim.rsi()
        
        
        dfw["kw"]=stoch_rsi_w.stochrsi_k()
        
        dfm["km"]=stoch_rsi_m.stochrsi_k()
        dfw["Chng"]=(dfw.Close-dfw.Open)/dfw.Open
        STD="{:.2%}".format(np.std(dfw.Chng))
       
        lw=len(dfw.iloc[:,0])
        lm=len(dfm.iloc[:,0])
        #ld=len(dfd.iloc[:,0])
    
        try:
            last=dfw['Close'][lw-1]
        except:
            last="na"
            
        try:
            kw="{:.2%}".format(dfw['kw'][lw-1])
        except:
            kw="na"
        
        try:
            km="{:.2%}".format(dfm['km'][lm-1])
        except: 
            km="na"
            
        efw=dfw['efi_w'][lw-1]
        ef_rsi_w=(dfw['efi_rsi_w'][lw-1])
    
        efm=dfm['efi_m'][lm-1]
        ef_rsi_m=dfm['efi_rsi_m'][lm-1]
        #print(dfm(tail())

        #print(kw,km)
        temp=pd.Series([stk,last,STD,kw,km,efw,ef_rsi_w,efm,ef_rsi_m])
        #print(temp)
        df_st_rsi=df_st_rsi.append(temp,ignore_index=True)
    df_st_rsi.columns=['stk','last','STD','kw','km','efw','ef_rsi_w','efm','ef_rsi_m']
    return(df_st_rsi)

def get_call():
    df_call=pd.DataFrame()
    for i in range(0,len(sip)):
    #for i in range(0,200):
        try:
            stk=sip[i]
            tick= yf.Ticker(stk)
            ch=tick.option_chain(date=friday).calls
            ch=ch[ch.inTheMoney==False]
            #print(ch)
            #ln=len(ch.iloc[:,0])
            yld="{:.2%}".format((ch.iloc[0,3])/ch.iloc[0,2])
            yld_adj="{:.2%}".format((ch.iloc[0,3])/dte*7/ch.iloc[0,2])
            strike=ch.iloc[0,2]
            premium=ch.iloc[0,3]
            premium_adj=ch.iloc[0,3]/dte*7
            temp=pd.Series([stk,strike,premium,premium_adj,yld,yld_adj])
            df_call=df_call.append(temp,ignore_index=True)
        except:
            pass
    df_call.columns=['stk','strike','premium','premium_adj.','yield','yield_adj']
    return(df_call)





#print(f1)


#print(f2)
#print(f1)

#print(df)
#print(f3)



In [10]:
f2=get_call()
print(f2)

      stk  strike  premium  premium_adj.  yield yield_adj
0     MMM   180.0     0.58          4.06  0.32%     2.26%
1     AMT   285.0     1.55         10.85  0.54%     3.81%
2     ABC   121.0     0.50          3.50  0.41%     2.89%
3     ADI   172.5     0.70          4.90  0.41%     2.84%
4     ADM    65.0     0.22          1.54  0.34%     2.37%
..    ...     ...      ...           ...    ...       ...
131  TSCO   212.5     0.40          2.80  0.19%     1.32%
132  ULTA   372.5     1.45         10.15  0.39%     2.72%
133    VZ    53.0     0.07          0.49  0.13%     0.92%
134  VIAC    37.0     0.32          2.24  0.86%     6.05%
135   WFC    52.0     0.07          0.49  0.13%     0.94%

[136 rows x 6 columns]


In [11]:
sip=f2['stk'].to_numpy()
f1=st_rsi()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [13]:
f3=f1.set_index('stk').join(f2.set_index('stk'))
f3.to_csv('trade_matrix.csv')